# 4.6 E se meus dados forem imagens?

**Nome: Joana de Medeiros Oliveira Hulse Molinete**

### Introdução:

Redes neurais convolucionais diferem das redes neurais tradicionais porque aplicam um tipo de aprendizado hierarquizado dos dados de entrada, ou seja, aprendem de forma gradual, dividindo a entrada recebida em setores que serão processados cada vez mais correlacionadamente. Por isso, uma rede neural convolucional é uma ótima abordagem para dados que possuem correlação, assim a CNN vai ter sua melhor performance. 

Uma rede neural convolucional trabalha de forma diferente de uma MLP normal, por exemplo, já que possui 3 principais formatos de camadas de neurônios, que são repetidas até a camada final. A primeira camada é a camada de convolução, onde cada neurônio vai ter uma pequena área do dado de entrada (imagem, vídeo, áudio, ...) que será "varrido" por um filtro (kernel) - uma matriz de dimensão pequena - e tem como objetivo identificar e extrair características da entrada. Após esse processo, chamado de convolução, a saída da camada será um mapa de características, ou seja, uma matriz 2d das informações extraídas da entrada.

Imediatamente depois da camada de convolução vem uma função de ativação, tipicamente ReLU, para implementar a não-linearidade aos dados de saída. Ao adicionarmos não-linearidade aos dados, garantimos que a rede modele relacionamentos complexos, distinguindo diferentes características, o que facilita o treinamento e melhora a robustez ao ruído.

Após a camada de ativação, temos a camada de agrupamento, que filtra a saída da camada de convolução com um kernel sem pesos associados, identificando as características principais da saída e reduzindo a dimensão dos dados. 

Essas camadas são intercaladas diversas vezes, até que por fim cheguem a camada densa (ou fully connected layer) da rede neural. Como o nome indica, todos os neurônios da camada anterior a ela são ligados a todos os neurônios da camada final, que achatam as pilhas de saída das camadas anteriores e realizam tarefas de classificação de acordo com todo o aprendizado acumulado até lá.

### Camada de convolução:

Os processos de uma CNN são realizados por meio de produtos de matrizes, em sua maioria. A função de uma camada de convolução é extrair padrões locais da imagem, que de acordo com o aprofundamento das camadas vão se relacionando cada vez mais, cobrindo toda a imagem de forma que a rede aprenda a identificar padrões globais.

A camada de convolução conta com alguns parâmetros importantes: o filtro, que é uma matriz quadrada de tamanho menor do que a entrada do neurônio, com pesos em cada posição, que são aprendidos e aprimorados durante o treinamento da rede; o stride, que nada mais é do que o número de pixels que o filtro se move por vez; e o número de filtros que serão aplicados na entrada, que nos dirá os padrões a serem aprendidos, e no fim, quantos mapas de ativação teremos como saída da camada.

A rede aprende a identificar padrões na entrada por meio de um processo chamado de convolução, que nada mais é do que deslocar o filtro por toda entrada, se movendo de acordo com o stride defininido. Como dito, o filtro é uma matriz quadrada com diferentes pesos, e a entrada de cada neurônio também será uma matriz, dessa forma, o filtro varre toda a entrada fazendo o produto elemento a elemento, que no fim serão somados a cada nova posição do filtro, fornecendo apenas um valor equivalente a aquela área em que estava posicionado. No fim, teremos uma matriz de mesmo tamanho que a entrada, com seus valores calculados de acordo com os pesos do filtro aplicado, que é chamado de mapa de ativação (feature map) com os dados tratados.

Após várias camadas de convolução, a rede começa a aprender padrões mais complexos da entrada e a cobrir áreas cada vez maiores da imagem, até que consiga correlacionar os dados antes separados. 

## Instalando os modos e bibliotecas necessárias:

In [1]:
import numpy as np
import torch
import torch.nn.functional as functional
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from torchmetrics import Precision
from tqdm import tqdm

## Definindo as camadas e funcionamento da nossa rede neural:

Note que aqui, usaremos a biblioteca PyTorch para definir, treinar e testar nossa CNN. A função de ativação que será utilizada é a Softmax [1], que transforma os logits (a saída da última camada da rede) em distribuições de probabilidade. Como estamos trabalhando com um problema de multiclassificação, a função Softmax é a mais adequada, já que vai não só indicar qual é a provável classe do dado (números de 0 a 9), mas também o quão "certa" a rede está sobre a decisão que será tomada, ou seja, ela vai retornar a distribuição de probabilidades entre todas as classes da rede.

In [2]:
class CNN(nn.Module):
    def __init__(self, num_dados_entrada, num_classes=10):
        super().__init__()
        
        self.camadas = nn.Sequential(
            # primeira camada de convolução:1 dado de entrada (a imagem), 8 dados de saída, kernel 3x3
            nn.Conv2d(in_channels=num_dados_entrada, out_channels=8, kernel_size=3, stride=1, padding=1),
            # função de ativação ReLU:
            nn.ReLU(),
            # primeira camada de agrupamento: usando Max pooling, com um kernel 2x2
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # segunda camada de convolução: 8 dados de entrada, 16 dados de saída, kernel 3x3
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # camada densa: dados de entrada com dimensões 16*7*7 depois de duas reduções por agrupamento (28->14->7), 10 possibilidades de saída (0 a 10)
        self.fc = nn.Linear(16 * 7 * 7, num_classes)
    
    def forward(self, x):
        x = self.camadas(x)
        x = torch.flatten(x, 1) # achata os dados para um vetor de tamanho 784 (a entrada precisa ser 1D) 
        x = self.fc(x)
        x = functional.softmax(x, dim=1)
        return x

**Será necessário utilizar o GPU da máquina para que o treinamento não demore muito. Abaixo, transferimos os dados para GPU, caso disponível, senão, continuamos utilizando a CPU.**

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Definindo os hiperparâmetros da nossa rede neural:**

In [4]:
entrada = 784 # 28x28 pixels
num_classes = 10 # 0-9
taxa_aprendizado = 0.001
tamanho_lote = 64
num_epocas = 60

**Importando o dataset que será utilizado:** 

Utilizaremos o dataset MNIST, muito popular para problemas de classificação multiclasse de imagens, já que possui uma base de dados de imagens de caracteres escritos a mão.

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
# normalizamos entre 0.1307 (média dos pixels do MNIST) e 0.3081 (desvio padrão dos pixels do MNIST)

dataset_treino = datasets.MNIST(root='dataset/', download=True, train=True, transform=transform)
carrega_dados_treino = DataLoader(dataset=dataset_treino, batch_size=tamanho_lote, shuffle=True)

dataset_teste = datasets.MNIST(root='dataset/', download=True, train=False, transform=transform)
carrega_dados_teste = DataLoader(dataset=dataset_teste, batch_size=tamanho_lote, shuffle=True)

In [6]:
minha_cnn = CNN(num_dados_entrada=1, num_classes=num_classes).to(device)

**Aqui, definimos a função de perda sendo a Perda de Entropia Cruzada (Cross Entropy Loss), visto que o problema abordado pelo algoritmo será de classificação.**

In [7]:
funcao_perda = nn.CrossEntropyLoss()
otimizador = optim.SGD(minha_cnn.parameters(), lr=taxa_aprendizado)

## Treinando a rede neural convolucional:

In [8]:
for epoca in range(num_epocas):
    minha_cnn.train()
    print(f'Epoca: [{epoca+1}/{num_epocas}]')
    for lote_index, (dados, y_true) in enumerate(tqdm(carrega_dados_treino)):
        # movendo os dados e os targets para o CPU ou GPU:
        dados = dados.to(device)
        y_true = y_true.to(device)
        
        # forward pass: 
        y_pred = minha_cnn(dados)
        
        # zero grad:
        otimizador.zero_grad()
        
        # loss:
        loss = funcao_perda(y_pred, y_true)
        
        # backpropagation:
        loss.backward()
        
        # atualiza parametros:
        otimizador.step()

Epoca: [1/60]


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:20<00:00, 45.06it/s]


Epoca: [2/60]


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:20<00:00, 46.49it/s]


Epoca: [3/60]


...

100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:21<00:00, 44.24it/s]


Epoca: [59/60]


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:20<00:00, 46.37it/s]


Epoca: [60/60]


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:21<00:00, 44.30it/s]


## Testando a rede neural:

Para identificar o desempenho da nossa CNN, vamos utilizar a métrica de erro de acurácia (média dos acertos obtidos) e precisão (média dos acertos obtidos em cada classe). 

In [9]:
acuracia = Accuracy(task = "multiclass", num_classes = 10)
precisao = Precision(task = "multiclass", num_classes = 10)

minha_cnn.eval()

with torch.no_grad():
    for imagem, rotulo in carrega_dados_teste:        
        output = minha_cnn(imagem)
        _, preds = torch.max(output, 1)
        acuracia(preds, rotulo)
        precisao(preds, rotulo)
               
testa_acuracia = acuracia.compute()
testa_precisao = precisao.compute()

print(f"Acurácia: {testa_acuracia}")

print(f"Precisão: {testa_precisao}")

Acurácia: 0.8679999709129333
Precisão: 0.8679999709129333


In [10]:
print(f"preds: shape={preds.shape}, dtype={preds.dtype}")
print(f"rotulo: shape={rotulo.shape}, dtype={rotulo.dtype}")

preds: shape=torch.Size([16]), dtype=torch.int64
rotulo: shape=torch.Size([16]), dtype=torch.int64


In [11]:
print(f"Valores únicos em preds: {preds.unique()}")
print(f"Valores únicos em rotulo: {rotulo.unique()}")

Valores únicos em preds: tensor([0, 1, 2, 4, 6, 7, 8, 9])
Valores únicos em rotulo: tensor([0, 1, 2, 4, 5, 6, 7, 9])


## Conclusão:

Notamos que a implementação e treinamento da nossa rede neural foi satisfatória ao analisarmos o desempenho da CNN, que apesar de não ter indicadores excelentes (acurácia e precisão abaixo de 90%), se saiu moderadamente bem. Os valores de 85% de acurácia e 85% de precisão indicam que a rede de fato aprendeu, não se apegou demais aos dados (não "overfittou") mas também generalizou moderadamente bem.

## Referências:



[1] What is the use of SoftMax in CNN?. Geeks for Geeks, 2024. Disponível em: https://www.geeksforgeeks.org/why-should-softmax-be-used-in-cnn/

[2] INGOLE, Mayur. Simple Convolutional Neural Network (CNN) for Dummies in PyTorch: A Step-by-Step Guide. Disponível em: https://medium.com/@myringoleMLGOD/simple-convolutional-neural-network-cnn-for-dummies-in-pytorch-a-step-by-step-guide-6f4109f6df80.

[3] LUNA, Javier Canales. PyTorch CNN Tutorial: Build and Train Convolutional Neural Networks in Python. Disponível em: https://www.datacamp.com/tutorial/pytorch-cnn-tutorial.

[4] Training a Classifier. Documentação PyTorch. Disponível em: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html.